# basic idea

consider each feature as a player and the dataset as a team.  
SHAP calculates the impact of every feature to the target variable (called shap value) using combinatorial calculus  
and retraining the model over all the combination of features that contains the one we are considering.  
The average absolute value of the impact of a feature against a target variable can be used as a measure of its importance.

# diff plots

## 1. summary plot

Summary plot is the plot of all shap values for all features and all values: 

<img src="imgs/sp.png" alt="sp" height="300"/>

Each point is a feature in a certain datum.  
Diff rows are same feature from diff data (record). The features are sorted from the most important one to the less important (s5 is the most important).  
In each row, higher the value of this feature, the more positive the impact on the target. The lower this value, the more negative the contribution.


## 2. force plot

Force plot show the shap value of all features in a single record (data)

<img src="imgs/fp.png" alt="fp" height="150"/>

predicted value: the output of the record (113.90 in pic).  
base value: is the average value of the target variable (outputs of model) across all the records (~108 in pic). 

Each stripe shows the impact of its feature in pushing the value of the target variable farther or closer to the base value.  
Red stripes show that their features push the value towards higher values. Blue, lower.  
The wider a stripe, the higher (in absolute value) the contribution.  
The sum of these contributions pushes the value of the target variable from the vase value to the final, predicted value.


# Example with sklearn neural network

In [ ]:
import shap
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPRegressor
from sklearn.pipeline import make_pipeline

from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split

'''data'''
X,y = load_diabetes(return_X_y=True)
features = load_diabetes()['feature_names']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

'''define and train model'''
model = make_pipeline(
    StandardScaler(),
    MLPRegressor(hidden_layer_sizes=(5,),activation='logistic', max_iter=10000,learning_rate='invscaling',random_state=0)
)

model.fit(X_train,y_train)

In [ ]:
"""shap"""

'''first get an explainer object'''
explainer = shap.KernelExplainer(model.predict,X_train)

'''
cal sharp value
shap values are calculated resampling the training dataset and calculating the impact over these perturbations, so need to define how much samples to resample

output: a (n_samples,n_features) numpy array. 
        Each element is the shap value of that feature of that record (shap values are calculated for each feature and for each record)
'''
shap_values = explainer.shap_values(X_test,nsamples=100)


'''do sharp plotting'''
''''summary plot'''
shap.summary_plot(shap_values,X_test,feature_names=features)

''''force plot'''
shap.initjs()
shap.force_plot(explainer.expected_value, shap_values[0,:]  ,X_test[0,:],feature_names=features)